---
# IMPORTANT

**Please remember to save this notebook `SC201L14.ipynb` as you work on it!**

### 請大家務必在這份檔案中使用 GPU。

請點選 `Runtime -> Change runtime type` 並將 `Hardware Accelerator` 設定為 `GPU`。

In [1]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# 請輸入 L14 資料夾之所在位置
FOLDERNAME = 'Colab\ Notebooks/SC201L14'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))

# this downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd drive/MyDrive/$FOLDERNAME/sc201/datasets/
!bash get_datasets.sh
%cd /content

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/SC201L14/sc201/datasets
/content


# What is PyTorch?

PyTorch 是一套計算系統，可以用來計算動態圖形 (neural network 是圖形的一種)。這些圖形是由 PyTorch 的 Tensor 物件組成的，Tensor 的用法如同 numpy 矩陣。PyTorch 內建自動微分的功能，使用者就不必手動處理 backward pass！

This notebook assumes that you are using **PyTorch version 1.4+**

## Why PyTorch?

* PyTorch 支援 GPU 計算，我們的 training 就可以利用 GPU 執行，程式會跑的更快！
* PyTorch 也是使用 modular design，大家以後就可以直接使用 PyTorch 既有模組（或是自己定義）並隨意拼湊成各式各樣的 neural network！
* 學術和業界中的 machine learning 都是使用 PyTorch 或是其他類似的強大計算套件，大家也就能跟上最新的研究和應用！

## How can I learn PyTorch on my own?

有興趣可以參考網路上的 PyTorch 教學，如 https://github.com/jcjohnson/pytorch-examples

另外也可以參考 PyTorch 的說明書 [API doc](http://pytorch.org/docs/stable/index.html)。PyTorch 相關問題會建議大家在 [PyTorch forum](https://discuss.pytorch.org/) 上發問，而非 StackOverflow。

# Section I. Preparation

大家在之前的作業裡做 data preparation 都是呼叫我們提供的程式。

PyTorch 內建的 `DataLoader` 和 `sampler` 類別可以將這個步驟自動化。詳細用法請參考以下的 code，特別是 data 的正規化 (normalization) 和分劃 (partitioning into *train / val / test*)。

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

In [3]:
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./sc201/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64,
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./sc201/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64,
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./sc201/datasets', train=False, download=True,
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

100%|██████████| 170498071/170498071 [00:05<00:00, 29637723.27it/s]


Extracting ./sc201/datasets/cifar-10-python.tar.gz to ./sc201/datasets
Files already downloaded and verified
Files already downloaded and verified


我們透由 `device` 啟用 PyTorch 的 GPU 功能。

（如果您未將 CUDA 開啟，`torch.cuda.is_available()` 會回傳 False，使 notebook 轉回 CPU mode。）

In [4]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


In [5]:
def train_part34(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.

    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for

    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss_function = nn.CrossEntropyLoss()
            loss = loss_function(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()

In [6]:
def check_accuracy_part34(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

# PyTorch Sequential API

### Sequential API: Two-Layer Network
以下是 two-layer fully connected network 的 `nn.Sequential` 範例，我們把內建的 layer 依序丟入，並使用同樣的 training loop 進行訓練。

大家在這裡不用做 hyperparameter tuning，但是在不做 tuning 的情況下，模型應該還是能在一個 epoch 之內達到 40% 以上的準確率。

In [7]:
# We need to wrap `flatten` function in a module in order to stack it
# in nn.Sequential

hidden_layer_size = 4000
learning_rate = 1e-2

model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(3 * 32 * 32, hidden_layer_size),
    nn.ReLU(),
    nn.Linear(hidden_layer_size, 10),
)

# you can use Nesterov momentum in optim.SGD
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                     momentum=0.9, nesterov=True)

train_part34(model, optimizer, epochs=3)

Iteration 0, loss = 2.3875
Checking accuracy on validation set
Got 140 / 1000 correct (14.00)

Iteration 100, loss = 1.7819
Checking accuracy on validation set
Got 339 / 1000 correct (33.90)

Iteration 200, loss = 1.6635
Checking accuracy on validation set
Got 401 / 1000 correct (40.10)

Iteration 300, loss = 1.7745
Checking accuracy on validation set
Got 408 / 1000 correct (40.80)

Iteration 400, loss = 1.9584
Checking accuracy on validation set
Got 402 / 1000 correct (40.20)

Iteration 500, loss = 1.8865
Checking accuracy on validation set
Got 402 / 1000 correct (40.20)

Iteration 600, loss = 1.6952
Checking accuracy on validation set
Got 456 / 1000 correct (45.60)

Iteration 700, loss = 1.6894
Checking accuracy on validation set
Got 451 / 1000 correct (45.10)

Iteration 0, loss = 1.5628
Checking accuracy on validation set
Got 453 / 1000 correct (45.30)

Iteration 100, loss = 1.6852
Checking accuracy on validation set
Got 454 / 1000 correct (45.40)

Iteration 200, loss = 1.8673
Check

### Sequential API: Three-Layer ConvNet
請大家使用 `nn.Sequential` 建立並訓練出一套 three-layer ConvNet，架構依舊是：

1. Convolutional layer (with bias) with 32 3x3 filters, with zero-padding of 1
2. ReLU
3. Convolutional layer (with bias) with 16 3x3 filters, with zero-padding of 1
4. ReLU
5. Fully-connected layer (with bias) to compute scores for 10 classes

訓練的方式請使用 stochastic gradient descent with Nesterov momentum 0.9。

大家在這裡不用做 hyperparameter tuning，但是在不做 tuning 的情況下，模型應該還是能在一個 epoch 之內達到 55% 以上的準確率。

In [8]:
model = None
optimizer = None

################################################################################
# TODO: Rewrite the 2-layer ConvNet with bias from Part III with the           #
# Sequential API.                                                              #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


#model = nn.Sequential(
#    # N x 3 x 32 x 32
#    nn.Conv2d(3,32,3,1,1),
#    nn.ReLU(),

#    # N x 32 x 32 x 32
#    nn.Conv2d(32,16,3,1,1),
#    nn.ReLU(),

#    # N x 16 x 32 x 32
#    nn.Flatten(),
#   nn.Linear(16 * 32 * 32,10)

#)

# with max pooling and BatchNorm2d
model = nn.Sequential(
    # N x 3 x 32 x 32
    # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
    nn.Conv2d(3,32,3,1,1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(2,2),
    # N x 32 x 16 x 16
    nn.Conv2d(32,16,3,1,1),
    nn.BatchNorm2d(16),
    nn.ReLU(),
    nn.MaxPool2d(2,2),
    # N x 16 x 8 x 8

    nn.Flatten(),
    nn.Linear(16 * 8 * 8,100),

    nn.ReLU(),
    nn.Linear(100,10)

)


#optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum = 0.9, nesterov=True)
optimizer = optim.Adam(model.parameters(), lr = 1e-3)


# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE
################################################################################

train_part34(model, optimizer, epochs=3)

Iteration 0, loss = 2.3021
Checking accuracy on validation set
Got 136 / 1000 correct (13.60)

Iteration 100, loss = 1.5323
Checking accuracy on validation set
Got 430 / 1000 correct (43.00)

Iteration 200, loss = 1.4150
Checking accuracy on validation set
Got 523 / 1000 correct (52.30)

Iteration 300, loss = 1.2364
Checking accuracy on validation set
Got 559 / 1000 correct (55.90)

Iteration 400, loss = 1.2008
Checking accuracy on validation set
Got 594 / 1000 correct (59.40)

Iteration 500, loss = 1.2820
Checking accuracy on validation set
Got 582 / 1000 correct (58.20)

Iteration 600, loss = 1.1548
Checking accuracy on validation set
Got 588 / 1000 correct (58.80)

Iteration 700, loss = 1.0785
Checking accuracy on validation set
Got 631 / 1000 correct (63.10)

Iteration 0, loss = 1.0637
Checking accuracy on validation set
Got 592 / 1000 correct (59.20)

Iteration 100, loss = 0.8304
Checking accuracy on validation set
Got 631 / 1000 correct (63.10)

Iteration 200, loss = 1.2584
Check

In [9]:
from torchsummary import summary
summary(model, (3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             896
       BatchNorm2d-2           [-1, 32, 32, 32]              64
              ReLU-3           [-1, 32, 32, 32]               0
         MaxPool2d-4           [-1, 32, 16, 16]               0
            Conv2d-5           [-1, 16, 16, 16]           4,624
       BatchNorm2d-6           [-1, 16, 16, 16]              32
              ReLU-7           [-1, 16, 16, 16]               0
         MaxPool2d-8             [-1, 16, 8, 8]               0
           Flatten-9                 [-1, 1024]               0
           Linear-10                  [-1, 100]         102,500
             ReLU-11                  [-1, 100]               0
           Linear-12                   [-1, 10]           1,010
Total params: 109,126
Trainable params: 109,126
Non-trainable params: 0
-------------------------------